<a href="https://colab.research.google.com/github/myspaceria/AZthesiscode/blob/main/PRISMmaxAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import requests
from datetime import datetime, timedelta

base_url = 'https://services.nacse.org/prism/data/get/us/4km/tmax'
output_dir = 'prism_maxdownload'
os.makedirs(output_dir, exist_ok=True)

start_date = datetime(2016, 5, 1)
end_date = datetime(2020, 9, 30)

current_date = start_date
while current_date <= end_date:
    # Only download if month between May (5) and Sept (9)
    if 5 <= current_date.month <= 9:
        date_str = current_date.strftime('%Y%m%d')
        url = f"{base_url}/{date_str}"
        print(f"Downloading {url}...")
        response = requests.get(url)
        if response.status_code == 200:
            filename = response.headers.get('content-disposition')
            if filename:
                # extract filename from header
                fname = filename.split('filename=')[1].strip('"')
            else:
                fname = f"PRISM_tmax_stable_4kmD1_{date_str}_bil.bil"
            with open(os.path.join(output_dir, fname), 'wb') as f:
                f.write(response.content)
        else:
            print(f"Failed to download for {date_str}, status: {response.status_code}")
    current_date += timedelta(days=1)


In [ ]:
!pip install rasterio

In [ ]:
import os
import zipfile

zip_folder = "prism_maxdownload/"
extract_folder = "prism_extract/"
os.makedirs(extract_folder, exist_ok=True)

# Unzip all zip files
for zip_filename in os.listdir(zip_folder):
    if zip_filename.endswith('.zip'):
        zip_path = os.path.join(zip_folder, zip_filename)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_folder)


In [ ]:
import os
import rasterio
import numpy as np

# Folder containing the extracted .tif PRISM files
input_folder = "/content/prism_extract/"
output_file = "/content/prism_max_temperature_all_years.tif"

# List all .tif files in the input folder
tif_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".tif")]

# Initialize variables
sum_data = None
count = 0

for idx, tif_path in enumerate(tif_files):
    with rasterio.open(tif_path) as src:
        data = src.read(1).astype(np.float32)
        nodata = src.nodata

        if nodata is not None:
            mask = data == nodata
            data[mask] = np.nan  # Handle no-data as NaN
        else:
            mask = np.isnan(data)

        if sum_data is None:
            sum_data = np.zeros_like(data, dtype=np.float32)

        sum_data += np.nan_to_num(data)
        count += (~mask).astype(int)  # Add only valid pixels

# Avoid divide by zero
mean_data = np.divide(sum_data, count, out=np.full_like(sum_data, np.nan), where=count != 0)

# Save the average image
with rasterio.open(tif_files[0]) as src0:
    profile = src0.profile
    profile.update(dtype=rasterio.float32, count=1, compress='lzw', nodata=np.nan)

    with rasterio.open(output_file, 'w', **profile) as dst:
        dst.write(mean_data.astype(np.float32), 1)

print(f"Saved mean raster: {output_file}")


